In [2]:
import RDatasets: dataset

ome_dataset = dataset("MASS", "OME")
ome_dataset = filter(row -> row.OME != "N/A", ome_dataset)
first(ome_dataset, 5)

Row,ID,Age,OME,Loud,Noise,Correct,Trials
,Int32,Int32,Cat…,Int32,Cat…,Int32,Int32
1,1,30,low,35,coherent,1,4
2,1,30,low,35,incoherent,4,5
3,1,30,low,40,coherent,0,3
4,1,30,low,40,incoherent,1,1
5,1,30,low,45,coherent,2,4


In [3]:
dat = copy(ome_dataset)
dat.Prob = dat.Correct ./ dat.Trials
dat.OMElow = ifelse.(dat.OME .== "low", 1, 0)
dat.NoiseCoherent = ifelse.(dat.Noise .== "coherent", 1, 0)
first(dat, 5)

Row,ID,Age,OME,Loud,Noise,Correct,Trials,Prob,OMElow,NoiseCoherent
,Int32,Int32,Cat…,Int32,Cat…,Int32,Int32,Float64,Int64,Int64
1,1,30,low,35,coherent,1,4,0.25,1,1
2,1,30,low,35,incoherent,4,5,0.8,1,0
3,1,30,low,40,coherent,0,3,0.0,1,1
4,1,30,low,40,incoherent,1,1,1.0,1,0
5,1,30,low,45,coherent,2,4,0.5,1,1


In [4]:
using GLM, DataFrames

model = glm(@formula(Prob ~ Age + OMElow + Loud + NoiseCoherent), dat, Binomial(), LogitLink(), wts=dat.Trials)

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, LogitLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

Prob ~ 1 + Age + OMElow + Loud + NoiseCoherent

Coefficients:
──────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error       z  Pr(>|z|)  Lower 95%    Upper 95%
──────────────────────────────────────────────────────────────────────────────
(Intercept)    -5.71813   0.416159    -13.74    <1e-42  -6.53379   -4.90248
Age             0.018896  0.00376638    5.02    <1e-06   0.011514   0.026278
OMElow         -0.23715   0.123228     -1.92    0.0543  -0.478672   0.00437225
Loud            0.171682  0.00887631   19.34    <1e-82   0.154284   0.189079
NoiseCoherent  -1.5763    0.1152      -13.68    <1e-41  -1.80209   -1.35052
──────────────────────────────────────────────────────────────────────────────

In [5]:
# Assuming model is already defined and trained
# Example: model = glm(@formula(Y ~ age + OMElow + Loud + NoiseCoherent), data, Normal(), IdentityLink())

new_data = DataFrame(Age = [60], OMElow = [0], Loud = [60], NoiseCoherent = [1])
predict(model, new_data)


1-element Vector{Union{Missing, Float64}}:
 0.9843301191352406

In [6]:
import CategoricalArrays: categorical, levelcode

dc = copy(dat)
dc.ID = categorical(dc.ID)  # creates dense categorical# dense integer labels starting from 1


unique(levelcode.(dc.ID))  # integer labels starting from 1

63-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
  ⋮
 52
 53
 54
 55
 56
 57
 58
 59
 60
 61
 62
 63

In [7]:
import LinearAlgebra: dot
import Turing: @model, sample, NUTS, Gibbs
import DistributionsAD: filldist

using Distributions
import StatsFuns: logistic

@model function prob_regression(X, y)
    μ ~ Normal(0, 10)
    τ² ~ InverseGamma(0.5, 0.5)

    a ~ filldist(Normal(μ, sqrt(τ²)), size(unique(X.ID), 1))
    b ~ filldist(Normal(0, 4), 4)

    ϕ = logistic.(a[X.level] .+ b[1] * X.Age .+ b[2] * X.OMElow .+ b[3] * X.Loud .+ b[4] * X.NoiseCoherent)

    for i in eachindex(y)
        y[i] ~ Binomial(X[i, :Trials], ϕ[i])
    end
end

dat.ID = categorical(dat.ID)
dat.level = levelcode.(dat.ID)

t_model = prob_regression(dat[!, Not(:Prob, :Correct)], dat.Correct)

DynamicPPL.Model{typeof(prob_regression), (:X, :y), (), (), Tuple{DataFrame, Vector{Int32}}, Tuple{}, DynamicPPL.DefaultContext}(Main.prob_regression, (X = 712×9 DataFrame
 Row │ ID    Age    OME   Loud   Noise       Trials  OMElow  NoiseCoherent  le ⋯
     │ Cat…  Int32  Cat…  Int32  Cat…        Int32   Int64   Int64          In ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ 1        30  low      35  coherent         4       1              1     ⋯
   2 │ 1        30  low      35  incoherent       5       1              0
   3 │ 1        30  low      40  coherent         3       1              1
   4 │ 1        30  low      40  incoherent       1       1              0
   5 │ 1        30  low      45  coherent         4       1              1     ⋯
   6 │ 1        30  low      45  incoherent       2       1              0
   7 │ 1        30  low      50  coherent         3       1              1
   8 │ 1        30  low      50  incoherent     

In [8]:
chains = sample(t_model, NUTS(), 10000)

┌ Info: Found initial step size
└   ϵ = 0.00625
Sampling: 100%|█████████████████████████████████████████| Time: 0:06:00


Chains MCMC chain (10000×83×1 Array{Float64, 3}):

Iterations        = 1001:1:11000
Number of chains  = 1
Samples per chain = 10000
Wall duration     = 378.96 seconds
Compute duration  = 378.96 seconds
parameters        = μ, τ², a[1], a[2], a[3], a[4], a[5], a[6], a[7], a[8], a[9], a[10], a[11], a[12], a[13], a[14], a[15], a[16], a[17], a[18], a[19], a[20], a[21], a[22], a[23], a[24], a[25], a[26], a[27], a[28], a[29], a[30], a[31], a[32], a[33], a[34], a[35], a[36], a[37], a[38], a[39], a[40], a[41], a[42], a[43], a[44], a[45], a[46], a[47], a[48], a[49], a[50], a[51], a[52], a[53], a[54], a[55], a[56], a[57], a[58], a[59], a[60], a[61], a[62], a[63], b[1], b[2], b[3], b[4]
internals         = n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size, lp, logprior, loglikelihood

Use `describe(chains)` for summary statistics and quantiles.


In [9]:
describe(chains)

Chains MCMC chain (10000×83×1 Array{Float64, 3}):

Iterations        = 1001:1:11000
Number of chains  = 1
Samples per chain = 10000
Wall duration     = 378.96 seconds
Compute duration  = 378.96 seconds
parameters        = μ, τ², a[1], a[2], a[3], a[4], a[5], a[6], a[7], a[8], a[9], a[10], a[11], a[12], a[13], a[14], a[15], a[16], a[17], a[18], a[19], a[20], a[21], a[22], a[23], a[24], a[25], a[26], a[27], a[28], a[29], a[30], a[31], a[32], a[33], a[34], a[35], a[36], a[37], a[38], a[39], a[40], a[41], a[42], a[43], a[44], a[45], a[46], a[47], a[48], a[49], a[50], a[51], a[52], a[53], a[54], a[55], a[56], a[57], a[58], a[59], a[60], a[61], a[62], a[63], b[1], b[2], b[3], b[4]
internals         = n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size, lp, logprior, loglikelihood

Summary Statistics
  parameters      mean       std      mcse   ess_bulk   ess_tail   

In [ ]:
import AxisArrays: axisvalues

axisvalues(chains.value)[2] |> collect |> print

In [130]:
model_vars = [:μ, :τ², Symbol("a[1]"), Symbol("a[2]"), Symbol("a[3]"), Symbol("a[4]"), Symbol("a[5]"), Symbol("a[6]"), Symbol("a[7]"), Symbol("a[8]"), Symbol("a[9]"), Symbol("a[10]"), Symbol("a[11]"), Symbol("a[12]"), Symbol("a[13]"), Symbol("a[14]"), Symbol("a[15]"), Symbol("a[16]"), Symbol("a[17]"), Symbol("a[18]"), Symbol("a[19]"), Symbol("a[20]"), Symbol("a[21]"), Symbol("a[22]"), Symbol("a[23]"), Symbol("a[24]"), Symbol("a[25]"), Symbol("a[26]"), Symbol("a[27]"), Symbol("a[28]"), Symbol("a[29]"), Symbol("a[30]"), Symbol("a[31]"), Symbol("a[32]"), Symbol("a[33]"), Symbol("a[34]"), Symbol("a[35]"), Symbol("a[36]"), Symbol("a[37]"), Symbol("a[38]"), Symbol("a[39]"), Symbol("a[40]"), Symbol("a[41]"), Symbol("a[42]"), Symbol("a[43]"), Symbol("a[44]"), Symbol("a[45]"), Symbol("a[46]"), Symbol("a[47]"), Symbol("a[48]"), Symbol("a[49]"), Symbol("a[50]"), Symbol("a[51]"), Symbol("a[52]"), Symbol("a[53]"), Symbol("a[54]"), Symbol("a[55]"), Symbol("a[56]"), Symbol("a[57]"), Symbol("a[58]"), Symbol("a[59]"), Symbol("a[60]"), Symbol("a[61]"), Symbol("a[62]"), Symbol("a[63]"), Symbol("b[1]"), Symbol("b[2]"), Symbol("b[3]"), Symbol("b[4]")]

size(model_vars)

(69,)

In [131]:
chains

Chains MCMC chain (10000×83×1 Array{Float64, 3}):

Iterations        = 1001:1:11000
Number of chains  = 1
Samples per chain = 10000
Wall duration     = 142.71 seconds
Compute duration  = 142.71 seconds
parameters        = μ, τ², a[1], a[2], a[3], a[4], a[5], a[6], a[7], a[8], a[9], a[10], a[11], a[12], a[13], a[14], a[15], a[16], a[17], a[18], a[19], a[20], a[21], a[22], a[23], a[24], a[25], a[26], a[27], a[28], a[29], a[30], a[31], a[32], a[33], a[34], a[35], a[36], a[37], a[38], a[39], a[40], a[41], a[42], a[43], a[44], a[45], a[46], a[47], a[48], a[49], a[50], a[51], a[52], a[53], a[54], a[55], a[56], a[57], a[58], a[59], a[60], a[61], a[62], a[63], b[1], b[2], b[3], b[4]
internals         = n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size, lp, logprior, loglikelihood

Use `describe(chains)` for summary statistics and quantiles.


In [145]:
chains.value[:, :, 1][1, :]

1-dimensional AxisArray{Float64,1,...} with axes:
    :var, [:μ, :τ², Symbol("a[1]"), Symbol("a[2]"), Symbol("a[3]"), Symbol("a[4]"), Symbol("a[5]"), Symbol("a[6]"), Symbol("a[7]"), Symbol("a[8]")  …  :hamiltonian_energy, :hamiltonian_energy_error, :max_hamiltonian_energy_error, :tree_depth, :numerical_error, :step_size, :nom_step_size, :lp, :logprior, :loglikelihood]
And data, a 83-element Vector{Float64}:
       3.3006340344066274
       0.8247767831837268
       4.916700883312494
       2.632120016057998
       3.7750716213826534
       3.0977799071192313
       2.5328069833711457
       3.862486702393878
       2.449254643831275
       2.9088595350625153
       5.203344020133028
       3.3030710543437856
       2.0691528121078226
       ⋮
       0.49750779855774985
 -260333.0804115341
  260377.5781362109
       0.004996866453438997
      Inf
       1.0
       1.0
       0.00182441151036913
       0.00182441151036913
 -260332.887769039
    -105.17408530784857
 -260227.71368373116

In [167]:
samples = chains.value[:, model_vars, 1]
p = samples[1, :]
a = p[ [Symbol("a[$i]") for i in 1:63]]

a[dat.level]

mean_params = mean(samples, dims=1)

2-dimensional AxisArray{Float64,2,...} with axes:
    :iter, 1:1:1
    :var, [:μ, :τ², Symbol("a[1]"), Symbol("a[2]"), Symbol("a[3]"), Symbol("a[4]"), Symbol("a[5]"), Symbol("a[6]"), Symbol("a[7]"), Symbol("a[8]")  …  Symbol("a[58]"), Symbol("a[59]"), Symbol("a[60]"), Symbol("a[61]"), Symbol("a[62]"), Symbol("a[63]"), Symbol("b[1]"), Symbol("b[2]"), Symbol("b[3]"), Symbol("b[4]")]
And data, a 1×69 Matrix{Float64}:
 3.30062  0.825145  4.91675  2.63185  3.77514  …  -1.28495  5.05817  2.57116

In [168]:
import Statistics: mean

function compute_DIC(dat, chains, log_likelihood, parameter_names)
    # Extract samples for the parameters
    samples = chains.value[:, parameter_names, 1]
    n_samples = size(samples, 1)

    # Compute deviance for each sample
    deviances = [-2 * log_likelihood(dat, samples[i, :]) for i in 1:n_samples]
    mean_deviance = mean(deviances)

    # Compute deviance at the mean of the parameters
    mean_params = mean(samples, dims=1)[1, :]
    deviance_at_mean = -2 * log_likelihood(dat, mean_params)

    # Compute DIC
    DIC = 2 * mean_deviance - deviance_at_mean
    return DIC, mean_deviance - deviance_at_mean
end

log_likelihood(dat, params) = begin
    b1, b2, b3, b4 = params[ [Symbol("b[$i]") for i in 1:4] ]
    a = params[ [Symbol("a[$i]") for i in 1:63] ]
    ϕ = logistic.(a[dat.level] .+ b1 .* dat.Age .+ b2 .* dat.OMElow .+ b3 .* dat.Loud .+ b4 .* dat.NoiseCoherent)
    sum(logpdf.(Binomial.(dat.Trials, ϕ), dat.Correct))
end

compute_DIC(dat, chains, log_likelihood, model_vars)



(519907.1791354293, 0.013566617912147194)

In [123]:
model_vars

69-element Vector{Symbol}:
 :μ
 :τ²
 Symbol("a[1]")
 Symbol("a[2]")
 Symbol("a[3]")
 Symbol("a[4]")
 Symbol("a[5]")
 Symbol("a[6]")
 Symbol("a[7]")
 Symbol("a[8]")
 Symbol("a[9]")
 Symbol("a[10]")
 Symbol("a[11]")
 ⋮
 Symbol("a[56]")
 Symbol("a[57]")
 Symbol("a[58]")
 Symbol("a[59]")
 Symbol("a[60]")
 Symbol("a[61]")
 Symbol("a[62]")
 Symbol("a[63]")
 Symbol("b[1]")
 Symbol("b[2]")
 Symbol("b[3]")
 Symbol("b[4]")

In [121]:
# how to pack variables in julia, explain me with this example [1, 3, 4, 5]
tau, mu, b4, b3, b2, b1, a... = sort(model_vars, rev=true)

sort!(a)

63-element Vector{Symbol}:
 Symbol("a[10]")
 Symbol("a[11]")
 Symbol("a[12]")
 Symbol("a[13]")
 Symbol("a[14]")
 Symbol("a[15]")
 Symbol("a[16]")
 Symbol("a[17]")
 Symbol("a[18]")
 Symbol("a[19]")
 Symbol("a[1]")
 Symbol("a[20]")
 Symbol("a[21]")
 ⋮
 Symbol("a[57]")
 Symbol("a[58]")
 Symbol("a[59]")
 Symbol("a[5]")
 Symbol("a[60]")
 Symbol("a[61]")
 Symbol("a[62]")
 Symbol("a[63]")
 Symbol("a[6]")
 Symbol("a[7]")
 Symbol("a[8]")
 Symbol("a[9]")

In [114]:
c

2-element Vector{Int64}:
 4
 5

In [110]:
model_vars

69-element Vector{Symbol}:
 :μ
 :τ²
 Symbol("a[1]")
 Symbol("a[2]")
 Symbol("a[3]")
 Symbol("a[4]")
 Symbol("a[5]")
 Symbol("a[6]")
 Symbol("a[7]")
 Symbol("a[8]")
 Symbol("a[9]")
 Symbol("a[10]")
 Symbol("a[11]")
 ⋮
 Symbol("a[56]")
 Symbol("a[57]")
 Symbol("a[58]")
 Symbol("a[59]")
 Symbol("a[60]")
 Symbol("a[61]")
 Symbol("a[62]")
 Symbol("a[63]")
 Symbol("b[1]")
 Symbol("b[2]")
 Symbol("b[3]")
 Symbol("b[4]")